<a href="https://colab.research.google.com/github/Anro607/memo_app/blob/main/doribasil_will_WIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2023학년도 교과융합 탐구 보고서 - 고전문학에 워투벡을 얹어드셔보세요(가제)
https://www.notion.so/ae8770f8f03e4028a746618d5074831c
<-내 노션바로가기

In [2]:
#웹 크롤
import requests

#전처리
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

#워드투벡
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# 1. 웹 크롤링, 문자열 인덱싱으로 전처리하기

In [ ]:
#웹 크롤링을 사용해 gutenberq에서 업로드한 도리온 그레이의 초상 원문 가져오기

url = "https://www.gutenberg.org/cache/epub/174/pg174.txt"  #Plain Text UTF-8
all_html = requests.get(url)                                #가져오기
print(all_html.text)                                        #제대로 가져와졌는지 확인

In [8]:
dorion = all_html.text                                      #전체 내용 텍스트

현재 dorion 변수에는 도리온 그레이의 초상 원문 뿐 아닌 구텐베르크 각주, 오스카와일드 작가의 말, 목차 등 분석에 불필요한 요소가 포함됨.

남겨야 할 것:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a. 본문

삭제해야 할 것:
1. 'The Project Gutenberg eBook of The Picture of Dorian Gray' ~ 'CHAPTER I.'
2. 'CHAPTER II', 'CHAPTER III' 등 I~XX 챕터 문구.
3. '  *** END OF THE PROJECT GUTENBERG EBOOK THE PICTURE OF DORIAN GRAY' ~ 끝까지

이를 a만 남기고, 본문 사이 2를 없애는 과정으로 어쩌구

In [ ]:
one = dorion.find('The studio was filled with the rich odour of roses,')                    #첫문장 (3314)
three = dorion.find('THE END')                                                              #디엔드 (438613)
print(one)
print(three)

고로 dorion 문자열 변수에서 0\~3314 인덱스와 438613\~끝 인덱스를 잘라야 하기에
반대로 3314\~438612 인덱스를 남길 것이다.

(438613이 아닌 438612을 잘라야 THE END의 'T'가 사라짐)

In [ ]:
a = dorion[3314: 438612]
print(a)

챕터 표기를 포함한 본문을 뽑아냄!

In [21]:
chapter = ['CHAPTER II.','CHAPTER III.','CHAPTER IV.','CHAPTER V.','CHAPTER VI.'
          ,'CHAPTER VII.','CHAPTER VIII.','CHAPTER IX.','CHAPTER X.','CHAPTER XI.'
          ,'CHAPTER XII.','CHAPTER XIII.','CHAPTER XIV.','CHAPTER XV.','CHAPTER XVI.',
           'CHAPTER XVII.','CHAPTER XVIII.','CHAPTER XIX.','CHAPTER XX.']                 #챕터리스트리스트 엌ㅋㅋ
b = a                                                                                     #원본 보호를 위한 임시 변수
for i in chapter:
  b = b.replace(i,'')                                                                     #제거

print(b.find('CHAPTER'))                                                                  #잘 제거됐는지 확인(-1은 해당 문자가 없다는 뜻!)

-1


필요한 요소만 남기는 일에 성공했으니 이제 진짜 토큰화를 해보자.

참고한 글 = https://wikidocs.net/21698